In [1]:
import os


In [2]:
%pwd

'/home/lab/Workspace/e2e-mlflow-dvc/research'

In [3]:
%cd /home/lab/Workspace/e2e-mlflow-dvc

/home/lab/Workspace/e2e-mlflow-dvc


In [4]:
#entity DataIngestionConfig 
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir :Path
    source_URL:str
    local_data_file: Path
    unzip_dir:Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from pathlib import Path


In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Now create the directories using the correct Path type
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Ensure that the root_dir is created as well
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [8]:
import gdown
from pathlib import Path 


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the URL
        '''
        try: 
            dataset_url = str(self.config.source_URL)  # Ensure it's a string
            zip_download_dir = str(self.config.local_data_file)  # Convert Path to string
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)  # Pass string path to gdown

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-28 12:04:00,995: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-28 12:04:01,000: INFO: common: YAML file loaded successfully: params.yaml]
[2025-01-28 12:04:01,002: INFO: common: Created directory at: artifacts]
[2025-01-28 12:04:01,005: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-01-28 12:04:01,006: INFO: 3682014447: Downloading data from https:/drive.google.com/file/d/148SCpWC3EO7phIZZoirIe9itUOgB1Xoe/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=148SCpWC3EO7phIZZoirIe9itUOgB1Xoe
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=148SCpWC3EO7phIZZoirIe9itUOgB1Xoe&confirm=t&uuid=df797025-3f0b-42aa-a570-a2406a5525df
To: /home/lab/Workspace/e2e-mlflow-dvc/artifacts/data_ingestion/data.zip
100%|████| 28.8M/28.8M [00:02<00:00, 9.87MB/s]

[2025-01-28 12:04:08,512: INFO: 3682014447: Downloaded data from https:/drive.google.com/file/d/148SCpWC3EO7phIZZoirIe9itUOgB1Xoe/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
